ALL THE EXTERNAL REUIQREMENTS ARE PRESNET IN ./requirement.txt

In [ ]:
%pip install -r requirement.txt

In [ ]:
#error can occur, careful!!
%pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_md-3.0.0/en_core_web_md-3.0.0.tar.gz
%pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bc5cdr_md-0.4.0.tar.gz

In [1]:
import os
cwd_path= os.getcwd()
cwd_path= cwd_path.replace("\\","/")

Lets scrape the required dataset 
Input will be a text file containing articles abstracts from pubmed

In [4]:
from my_packages.get_document import module_get_document

kg_disease= str(input("please enter the disease you want a knowledge graph for: "))
no_of_articles= int(input("please enter the number of articles you want to consider: ")) #expected wall time for 10 articles around 12 mins
dataset_file_path= cwd_path + "/datasets/dataset.txt"

module_get_document.get_docx(kg_disease, no_of_articles, dataset_file_path)


Search results saved to a text file.


Lets crete a log file for ouputs 

In [5]:
log_file_path= cwd_path + "/ouput/output_log.txt"

try:
    with open(log_file_path, 'w') as fh:
        fh.write("welcome to the log file!")
except Exception as e:
    print(e)

Lets create a dataframe that can be later deployed to neo4J

In [6]:
from my_packages.get_knowledgeGraph import module_getKnowledgeGrpah

output_df_path= module_getKnowledgeGrpah.get_gph(cwd_path, log_file_path, dataset_file_path)


d:\VIISem\knowledgeGraphUsingMondo\.conda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
100%|██████████| 122/122 [03:55<00:00,  1.93s/it]


Lets deploy df to neo4J :)

In [7]:
from my_packages.create_graph import module_createGraph
#the load_gh fn takes input of bolt url and password of sandbox, create your own sandbow on neo4J
class_labels= module_createGraph.load_gh("bolt://44.202.158.190:7687", "autos-volumes-correction", output_df_path)

{'total': 118, 'batches': 1, 'time': 5.584398984909058}
{'total': 0, 'batches': 1, 'time': 3.4603235721588135}
{'total': 0, 'batches': 1, 'time': 1.9675333499908447}
{'total': 0, 'batches': 1, 'time': 0.8836343288421631}


* Ignore below if you dont need to check efficiency!!

**calculating the efficiency of 5 abastract indivially**
- manually abstracting entites from each abstact
- creating a list of list
- chcking acciracy, no of enties found, no of entites not found
- returning a dataframe

In [3]:
from check_efficiency import module_get_efficiency, get_data
df= []
dataset, entities_list= get_data.dataset, get_data.entities_list

for i in range(len(dataset)):
    accuracy= module_get_efficiency.check_ef(dataset[i], entities_list[i])
    df.append(accuracy)

d:\VIISem\knowledgeGraphUsingMondo\.conda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [6]:
import pandas as pd
df1= pd.DataFrame(df, columns= ['Precsion', 'Recall', 'F1_Score'], index=None)
df1['Article_no'] = range(1, len(df) + 1)
df1.set_index('Article_no', inplace=True)

accuracy_score_output= cwd_path+ "/ouput/accuracy_score.csv"
try:
    with open(accuracy_score_output, 'w') as fh:
        fh.write()
except Exception as e:
    print(e)
    
df1.to_csv(accuracy_score_output)

TextIOWrapper.write() takes exactly one argument (0 given)
